In [1]:
import json
import config
import sys
import pandas as pd

In [2]:
def get_keys(path):
    with open(path) as f:
        return json.load(f)

In [3]:
keys = get_keys("/Users/adinasteinman/.secret/spotify_api.json")
client_id = keys['client_id']
client_secret = keys['client_secret']

In [4]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id=client_id,
                                                           client_secret=client_secret))



In [69]:
#Biggest playlist ever 
hugeplaylist = sp.user_playlist_tracks(playlist_id="54nv8jbrm4JoHEZ49Qvjgl")

In [20]:
#Preview the release date of one song in the playlist
hugeplaylist['items'][40]['track']['album']['release_date']

'2011-05-10'

## Functions

In [40]:
def analyze_playlist(creator, playlist_id):
    
    # Create empty dataframe
    playlist_features_list = ["artist","album","track_name",  "track_id","danceability","energy","key","loudness","mode", "speechiness","instrumentalness","liveness","valence","tempo", "duration_ms","time_signature"]
    
    playlist_df = pd.DataFrame(columns = playlist_features_list)
    
    # Loop through every track in the playlist, extract features and append the features to the playlist df
    
    playlist = sp.user_playlist_tracks(creator, playlist_id)["items"]
    for track in playlist:
        # Create empty dict
        playlist_features = {}
        # Get metadata
        playlist_features["artist"] = track["track"]["album"]["artists"][0]["name"]
        playlist_features["album"] = track["track"]["album"]["name"]
        playlist_features['release_date'] = track["track"]["album"]["release_date"]
        playlist_features["track_name"] = track["track"]["name"]
        playlist_features["track_id"] = track["track"]["id"]
        
        # Get audio features
        audio_features = sp.audio_features(playlist_features["track_id"])[0]
        for feature in playlist_features_list[4:]:
            playlist_features[feature] = audio_features[feature]
        
        # Concat the dfs
        track_df = pd.DataFrame(playlist_features, index = [0])
        playlist_df = pd.concat([playlist_df, track_df], ignore_index = True)
        
    return playlist_df

In [26]:
playlist_creator = "spotify"
playlist_id = "54nv8jbrm4JoHEZ49Qvjgl"

In [41]:
analyze_playlist(playlist_creator, playlist_id)

TypeError: list indices must be integers or slices, not str

## Offset / Pagination

In [74]:
p1 = sp.user_playlist_tracks(playlist_id="54nv8jbrm4JoHEZ49Qvjgl", offset=200)['items']

In [79]:
type(p1)

list

In [80]:
finalplaylist = [] 
for i in range(0,1000):
    if i % 100 == 0:
        finalplaylist.append(sp.user_playlist_tracks(playlist_id="54nv8jbrm4JoHEZ49Qvjgl", offset=i)['items'])

In [99]:
finalplaylist[0][0]['track']['album']['name']

'Katy Perry - Teenage Dream: The Complete Confection'

In [104]:
finalplaylist[0][0]['track']['name']

'Firework'

In [105]:
def analyze_playlist(playlistname):
    
    # Create empty dataframe
    playlist_features_list = ["artist","album","track_name",  "track_id","danceability","energy","key","loudness","mode", "speechiness","instrumentalness","liveness","valence","tempo", "duration_ms","time_signature"]
    
    
    # Loop through every track in the playlist, extract features and append the features to the playlist df
    
    for i in playlistname:
        # Create empty dict
        playlist_features = {}
        # Get metadata
        playlist_features["artist"] = i[i]["track"]["album"]["artists"][0]["name"]
        playlist_features["album"] = i[i]["track"]["album"]["name"]
        playlist_features['release_date'] = i[i]["track"]["album"]["release_date"]
        playlist_features["track_name"] = i[i]["track"]["name"]
        playlist_features["track_id"] = i[i]["track"]["id"]
        
        
        # Concat the dfs
        track_df = pd.DataFrame(playlist_features, index = [0])
        
        
    return track_df

In [106]:
analyze_playlist(hugeplaylist)

TypeError: string indices must be integers